In [2]:
import numpy as np
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from treeple.datasets import make_trunk_classification
import ydf
import matplotlib.pyplot as plt
from treeple import ObliqueRandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap
from treeple._lib.sklearn.tree._criterion import Gini
from treeple.tree._oblique_splitter import BestObliqueSplitterTester
from treeple.datasets import make_trunk_classification
import pandas as pd



In [14]:
def plot_proj_existing_data(X, y,
                max_features=10, 
                feature_combinations=1.5,
                plot_fig=True,
                random_state=1):

    criterion = Gini(1, np.array((0, 1)))

    min_samples_leaf = 1
    min_weight_leaf = 0.0
    random_state = np.random.RandomState(random_state)
    n_samples= X.shape[0]
    n_features= X.shape[1]

    #feature_combinations = 3.0
    monotonic_cst = None
    missing_value_feature_mask = None

    # X, y = make_trunk_classification(n_samples=n_samples, n_dim=n_features, n_informative=600, seed=0)
    y = y.reshape(-1,1).astype(np.float64)
    X= X.astype(np.float32)

    sample_weight = np.ones(n_samples)

    splitter = BestObliqueSplitterTester(
        criterion,
        max_features,
        min_samples_leaf,
        min_weight_leaf,
        random_state,
        monotonic_cst,
        feature_combinations,
    )
    splitter.init_test(X, y, sample_weight, missing_value_feature_mask)


    projection_matrix = splitter.sample_projection_matrix_py()
    print("projection matrix shape:", projection_matrix.shape)


    if plot_fig:
        # Visualize the projection matrix
        cmap = ListedColormap(["orange", "white", "green"])

        # Create a heatmap to visualize the indices
        fig, ax = plt.subplots(figsize=(6, 6))

        ax.imshow(projection_matrix, cmap=cmap, aspect=n_features / max_features, interpolation="none")

        ax.set(title="Sampled Projection Matrix", xlabel="Feature Index", ylabel="Projection Vector Index")
        ax.set_xticks(np.arange(n_features))
        ax.set_yticks(np.arange(max_features))
        ax.set_yticklabels(np.arange(max_features, dtype=int) + 1)
        ax.set_xticklabels(np.arange(n_features, dtype=int) + 1)

        # Create a mappable object
        sm = ScalarMappable(cmap=cmap)
        sm.set_array([])  # You can set an empty array or values here

        # Create a color bar with labels for each feature set
        colorbar = fig.colorbar(sm, ax=ax, ticks=[0, 0.5, 1], format="%d")
        colorbar.set_label("Projection Weight")
        colorbar.ax.set_yticklabels(["-1", "0", "1"])

        plt.show()

    print(f"n_feature = ", n_features, 
          "\nn_samples = ", n_samples, 
          "\nmax_features = ", max_features, 
          "\nfeature_combinations = ", feature_combinations)
    print("max_features * feature_combinations = ", max_features * feature_combinations)
    print("Number of non-zeros: ",len(projection_matrix.nonzero()[0]))
    return projection_matrix, len(projection_matrix.nonzero()[0])



In [11]:
def prepare_dataframe(X, y):
    df = pd.DataFrame(X)
    df.columns = [str(i) for i in df.columns]  # Convert column names to strings
    df["target"] = y.astype(int)  # Append target column

    return df

Initialize parameter settings

In [ ]:
# Shared hyperparameters that used for both models
MAX_DEPTH = 10
N_ESTIMATORS = 300
RANDOM_SEED = 42
N_JOBS=-1
BOOTSTRAP = True
MAX_FEATURE = 100
FEATURE_COMBINATIONS = 3.0

### ydf ###
params_ydf = dict()
params_ydf["label"] = "target"
params_ydf["max_depth"] = MAX_DEPTH
params_ydf["num_trees"] = N_ESTIMATORS
params_ydf["random_seed"] = RANDOM_SEED
params_ydf["honest"] = False
params_ydf["bootstrap_size_ratio"] = 1.0
params_ydf["bootstrap_training_dataset"] = BOOTSTRAP
params_ydf["categorical_algorithm"] = "ONE_HOT"
params_ydf["compute_oob_performances"] = False  #oob_score
params_ydf["compute_oob_variable_importances"] = False  ## oob_score
params_ydf["in_split_min_examples_check"] = True
params_ydf["min_examples"] = 1    #min_samples_leaf
params_ydf["keep_non_leaf_label_distribution"] = True   # not sure

params_ydf["max_num_nodes"] = 30

params_ydf["growing_strategy"] = "BEST_FIRST_GLOBAL"
# params_ydf["num_candidate_attributes"] = -1  #0 equivalent to sqrt max_features in treeple
params_ydf["num_candidate_attributes_ratio"] = 1.0
params_ydf["sorting_strategy"] = "IN_NODE" ###
# sparse oblique params
params_ydf["split_axis"] = "SPARSE_OBLIQUE"
params_ydf["sparse_oblique_weights"] = "BINARY"
params_ydf["sparse_oblique_projection_density_factor"] = FEATURE_COMBINATIONS
params_ydf["sparse_oblique_normalization"] = "MIN_MAX"
params_ydf["sparse_oblique_max_num_projections"] = MAX_FEATURE # modify this, find in sourse code in treeple
params_ydf["sparse_oblique_num_projections_exponent"] = 1.0   #2.0
params_ydf["sampling_with_replacement"] = False

### treeple ObliqueRandomForestClassifier ###
params_treeple = {}
params_treeple["n_estimators"] = N_ESTIMATORS
params_treeple["criterion"] = "entropy"
params_treeple["max_depth"] = MAX_DEPTH
params_treeple["min_samples_split"] = 2
params_treeple["min_samples_leaf"] = 1
params_treeple["min_weight_fraction_leaf"] = 0.0
params_treeple["max_features"] = MAX_FEATURE
params_treeple["max_leaf_nodes"] = 30
params_treeple["min_impurity_decrease"] = 0.0
params_treeple["bootstrap"] = BOOTSTRAP
params_treeple["oob_score"] = False
params_treeple["n_jobs"] = N_JOBS
params_treeple["random_state"] = RANDOM_SEED
params_treeple["verbose"] = 0
params_treeple["warm_start"] = False
params_treeple["class_weight"] = None
params_treeple["max_samples"] = None
params_treeple["feature_combinations"] = FEATURE_COMBINATIONS


In [ ]:
def constant_nNonzeros_simulation(n_tree,
                    n_samples, 
                    params_treeple, 
                    params_ydf,
                    n_non_zeros,
                    n_rows,
                    n_columns,
                    n_rep=2, 
                    plot=False):



    accs_ydf = np.zeros((len(n_columns), len(n_rows)))
    times_ydf = np.zeros(accs_ydf.shape)
    accs_treeple = np.zeros(accs_ydf.shape)
    times_treeple = np.zeros(accs_ydf.shape)

    # copy the params to avoid overwriting
    params_treeple1 = params_treeple.copy() 
    params_ydf1 = params_ydf.copy()
    

    for i, n_column in enumerate(n_columns):
        # n_column matches number of features
        max_features = n_column
        X, y = make_trunk_classification(n_samples=n_samples, n_dim=n_column, n_informative=600, seed=0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        df_train = prepare_dataframe(X_train, y_train)
        df_test = prepare_dataframe(X_test, y_test)


        for j, n_row in enumerate(n_rows):
            # n_rows matches number of projections
            n_dim= n_column

            params_treeple1["max_features"] = int(n_row)
            params_treeple1["feature_combinations"] = n_row

            params_ydf1["sparse_oblique_max_num_projections"] = int(n_row)
            params_ydf1["Sparse_oblique_projection_density_factor"] = n_row
            
            _, treeple_n_nonzeros = plot_proj_existing_data(X, y,
                max_features=10, 
                feature_combinations=1.5,
                plot_fig=False,
                random_state=1)

            print("=================================================")        
            print(f"n_dim: {n_dim} | n_tree: {n_tree} | n_samples: {n_samples}")
            print(f"n_row: {n_row} | n_column: {n_column} | n_non_zeros: {n_non_zeros}")

            
            acc_temp_ydf=0
            time_temp_ydf=0

            acc_temp_treeple=0
            time_temp_treeple=0

            f1_temp=0

            for _ in range(n_rep):
                # --- Train YDF ---
                learner = ydf.RandomForestLearner(**params_ydf1)
                start_time = time.time()
                ydf_model = learner.train(df_train)
                time_ydf = time.time() - start_time
                pred_ydf = ydf_model.predict(df_test)
                pred_ydf = (pred_ydf >= 0.5).astype(int) 

                acc_ydf = accuracy_score(y_test, pred_ydf)
                acc_temp_ydf+=acc_ydf
                time_temp_ydf+=time_ydf
            
                # --- Train Treeple ---
                treeple_model = ObliqueRandomForestClassifier(**params_treeple1)
                acc_treeple, time_treeple, pred_treeple = train_and_evaluate(treeple_model, "Treeple", X_train, X_test, y_train, y_test)
                acc_temp_treeple+=acc_treeple
                time_temp_treeple+=time_treeple

                # Calculate F1 score for two predictions
                f1_compare = f1_score(pred_ydf, pred_treeple)
                f1_temp+=f1_compare

            accs_ydf.append(acc_temp_ydf/n_rep)
            times_ydf.append(time_temp_ydf/n_rep)

            accs_treeple.append(acc_temp_treeple/n_rep)
            times_treeple.append(time_temp_treeple/n_rep)

            f1_scores.append(f1_temp/n_rep)

    return accs_ydf, times_ydf, accs_treeple, times_treeple, f1_scores

Initialize n_rows and n_columns

In [10]:
n_rows = np.ones(6)
n_rows[0] = 128
n_rows.shape
for i in range(0,n_rows.shape[0]-1):
    n_rows[i+1] = n_rows[i]*2
n_columns = n_rows